In [1]:
import pandas as pd

import experiment.utils.query as query_utils
from experiment.utils import transformation

In [2]:
# read the csv file
raw_ct_brain = transformation.get_project_root() / "tmp" / "data" / "raw_ct_brain.csv"
df = pd.read_csv(raw_ct_brain)
df["INDEX"] = df["PROTOKOL_NO"]
df.set_index('INDEX')

# get the db engine
engine = query_utils.get_db_engine()

In [10]:
# upsert values
df.to_sql(
    name="reports",
    schema="raw_data",
    con=engine,
    index=False,
    if_exists="replace",
    chunksize=1000,
    method='multi'
)

32132